### Pidilite - library Import

In [55]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime

In [56]:
# !pip install oauth2client

In [57]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

# Open Sheet
sheet = client.open_by_key("1uzbZWiVA_9C4WTKfU61AS8zKxKpQ3Q9lSn3khUf92iw")
PJP = sheet.worksheet("PJP")
Coverage_sh = sheet.worksheet("Coverage")

# Convert to DataFrame
PlannedPJP = get_as_dataframe(PJP)

In [58]:
# SQL Server connection details
server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

### Free Visibility

In [59]:
# Connect and fetch into DataFrame - Extrating Data from ISPStoreAttendance_NormalExport ( Test & Supervisor Filtered)
try:
    conn = pyodbc.connect(conn_str)
    print("✅ Connected to SQL Server")

    # Define your query
    query = ''' SELECT TRY_CONVERT(DATE, [SubmittedDate&Time],105) AS [Visit Date], USERNAME AS [Emp User ID],
                       FullName AS [Employee Name],
                       LISStoreCode AS StoreCode,
                       StoreName,
					   [IsAvailable],
					   COALESCE([NonAvailabilityReason],'') AS [NonAvailabilityReason],
					   [VisibilityType],
					   [VisibilityBrand],
					   [Image1Link]
                       ,[Image2Link]
                       ,[Image3Link]
                       FROM [PowerBI_LISV2].[PidiliteManagement].[AdhocVisibilityReportExport]
                       WHERE  [Month] = MONTH(GETDATE())
                       AND year=YEAR(GETDATE())
                       and [UserName] NOT LIKE '%SUP%' 
                       AND [UserName] NOT LIKE '%test%'
                       AND [UserName] NOT LIKE 'TEST%'
    '''
    # Fetch results into a DataFrame
    FreeVisibility = pd.read_sql(query, conn)
    # Close the connection
    conn.close()
except Exception as e:
    print(f"❌ Error: {e}")

✅ Connected to SQL Server


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_20424\3769006297.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  FreeVisibility = pd.read_sql(query, conn)


In [6]:
FreeVisibility.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1276 entries, 0 to 1275
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Visit Date             1276 non-null   object
 1   Emp User ID            1276 non-null   object
 2   Employee Name          1276 non-null   object
 3   StoreCode              1276 non-null   object
 4   StoreName              1276 non-null   object
 5   IsAvailable            1276 non-null   object
 6   NonAvailabilityReason  1276 non-null   object
 7   VisibilityType         1229 non-null   object
 8   VisibilityBrand        1229 non-null   object
 9   Image1Link             1276 non-null   object
 10  Image2Link             1276 non-null   object
 11  Image3Link             1276 non-null   object
dtypes: object(12)
memory usage: 119.8+ KB


In [60]:
FreeVisibility['ProductCode']=FreeVisibility['VisibilityType']+ FreeVisibility['VisibilityBrand']

In [61]:
FreeVisibility.columns

Index(['Visit Date', 'Emp User ID', 'Employee Name', 'StoreCode', 'StoreName',
       'IsAvailable', 'NonAvailabilityReason', 'VisibilityType',
       'VisibilityBrand', 'Image1Link', 'Image2Link', 'Image3Link',
       'ProductCode'],
      dtype='object')

In [62]:
FreeVisibility = FreeVisibility [['Visit Date', 'Emp User ID', 'Employee Name', 'StoreCode', 'StoreName',
       'IsAvailable', 'NonAvailabilityReason', 'ProductCode','VisibilityType',
       'VisibilityBrand', 'Image1Link', 'Image2Link', 'Image3Link']]

In [63]:
FreeVisibility.head(2)

Visit Date  Emp User ID Employee Name     StoreCode      StoreName  \
0  2025-05-01  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   
1  2025-05-01  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   

  IsAvailable NonAvailabilityReason       ProductCode VisibilityType  \
0         Yes                        Side CapFevicryl       Side Cap   
1         Yes                         End CapFevikwik        End Cap   

  VisibilityBrand                                         Image1Link  \
0        Fevicryl  https://lisimagenewv3.loveinstore.com//LISImag...   
1        Fevikwik  https://lisimagenewv3.loveinstore.com//LISImag...   

  Image2Link Image3Link  
0                        
1

In [64]:
FreeVisibility.to_csv(r'C:\Users\Administrator\LIS\Pidilite\FreeVisibility.csv', index=False)

### SOS Table

In [65]:
# Connect and fetch into DataFrame - Extrating Data from ISPStoreAttendance_NormalExport ( Test & Supervisor Filtered)
try:
    conn = pyodbc.connect(conn_str)
    print("✅ Connected to SQL Server")

    # Define your query
    query = ''' SELECT TRY_CONVERT(DATE, [SubmittedDate&Time]) AS [Visit Date]
                     ,[UserName] AS [Emp User ID]
                     ,[FullName] AS [Employee Name]
                     ,[LISStoreCode] AS [StoreCode]
                     ,[StoreName]
                     ,[CategoryTrackingName] AS Category
                     ,[CategoryProductName] AS [Category Product]
                     ,TgtCatSOS AS [SOS Target]
                     ,[Facings] AS [Facing Prod],
                     ,[TotalFacings] AS [Facing Total]
              FROM [PowerBI_LISV2].[PidiliteManagement].[CategoryTrackingVisitExport]
              WHERE  [Month] = MONTH(GETDATE())
                       AND year=YEAR(GETDATE())
                       and [UserName] NOT LIKE '%SUP%' 
                       AND [UserName] NOT LIKE '%test%'
                       AND [UserName] NOT LIKE 'TEST%'
    '''
    # Fetch results into a DataFrame
    SOSTable = pd.read_sql(query, conn)
    # Close the connection
    conn.close()
except Exception as e:
    print(f"❌ Error: {e}")

✅ Connected to SQL Server
❌ Error: Execution failed on sql ' SELECT TRY_CONVERT(DATE, [SubmittedDate&Time]) AS [Visit Date]
                     ,[UserName] AS [Emp User ID]
                     ,[FullName] AS [Employee Name]
                     ,[LISStoreCode] AS [StoreCode]
                     ,[StoreName]
                     ,[CategoryTrackingName] AS Category
                     ,[CategoryProductName] AS [Category Product]
                     ,TgtCatSOS AS [SOS Target]
                     ,[Facings] AS [Facing Prod],
                     ,[TotalFacings] AS [Facing Total]
              FROM [PowerBI_LISV2].[PidiliteManagement].[CategoryTrackingVisitExport]
              WHERE  [Month] = MONTH(GETDATE())
                       AND year=YEAR(GETDATE())
                       and [UserName] NOT LIKE '%SUP%' 
                       AND [UserName] NOT LIKE '%test%'
                       AND [UserName] NOT LIKE 'TEST%'
    ': ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Se

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_20424\443316406.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  SOSTable = pd.read_sql(query, conn)


In [66]:
SOSTable.head(5)

Visit Date  Emp User ID Employee Name     StoreCode      StoreName  \
0  2025-05-01  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   
1  2025-05-01  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   
2  2025-05-01  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   
3  2025-05-01  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   
4  2025-05-01  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   

                   Category          Category Product SOS Target Facing Prod  \
0                  Car Care                  Car Care         20          48   
1                Cera Clean                Cera Clean          6           3   
2                  DIY Kits                  DIY Kits         10           5   
3                Fevicol MR                Fevicol MR         32          50   
4  Fevicryl- Acrylic Colors  Fevicryl- Acrylic Colors         22          25   

  Facing Total  
0           85  
1           40  
2           25  
3           50  
4           45

In [67]:
SOSTable['Facing Prod']= SOSTable['Facing Prod'].fillna(0)

In [68]:
SOSTable=SOSTable.drop_duplicates()

In [69]:
SOSTable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3289 entries, 0 to 3288
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Visit Date        3289 non-null   object
 1   Emp User ID       3289 non-null   object
 2   Employee Name     3289 non-null   object
 3   StoreCode         3289 non-null   object
 4   StoreName         3289 non-null   object
 5   Category          3289 non-null   object
 6   Category Product  3289 non-null   object
 7   SOS Target        3289 non-null   object
 8   Facing Prod       3289 non-null   object
 9   Facing Total      3289 non-null   object
dtypes: object(10)
memory usage: 257.1+ KB


In [70]:
SOSTable.to_csv(r'C:\Users\Administrator\LIS\Pidilite\SOSTable.csv', index=False)

### MSL Data

In [42]:
# Connect and fetch into DataFrame - Extrating Data from ISPStoreAttendance_NormalExport ( Test & Supervisor Filtered)
try:
    conn = pyodbc.connect(conn_str)
    print("✅ Connected to SQL Server")

    # Define your query
    query = ''' SELECT  [SubmittedDate&Time] AS [Visit Date Time]
                ,[Username] AS [Emp User ID]
                ,[FullName] AS [Employee Name]
                ,[LISStoreCode] AS [StoreCode]
                ,[StoreName]  
                ,[ProductCode] AS [Product Code]
                ,[ProductName] AS [Product Name]
                  ,CASE WHEN [MSLStatus] = 'MSL' THEN 'Yes'ELSE 'No' END AS [MSL]
                  ,[ProductStatus] AS [OSA Satus]
                  ,Quantity AS [Quantity]
                  ,[Amount] AS [Amount(INR)]
                  ,CASE WHEN TRY_CAST([Quantity] AS float) IS NOT NULL AND TRY_CAST([Quantity] AS float) != 0
                        THEN TRY_CAST([Amount] AS float) / TRY_CAST([Quantity] AS float)
                        ELSE NULL
                    END AS [Cost(INR)]
                FROM [PowerBI_LISV2].[PidiliteManagement].[PrimaryShelfStockReportExport]
                WHERE  [Month] = MONTH(GETDATE())
                       AND year=YEAR(GETDATE())
                       and [UserName] NOT LIKE '%SUP%' 
                       AND [UserName] NOT LIKE '%test%'
                       AND [UserName] NOT LIKE 'TEST%'
    '''
    # Fetch results into a DataFrame
    MSLTable = pd.read_sql(query, conn)
    # Close the connection
    conn.close()
except Exception as e:
    print(f"❌ Error: {e}")

✅ Connected to SQL Server


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_20424\4101948083.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  MSLTable = pd.read_sql(query, conn)


In [44]:
MSLTable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16433 entries, 0 to 16432
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Visit Date Time  16433 non-null  datetime64[ns]
 1   Emp User ID      16433 non-null  object        
 2   Employee Name    16433 non-null  object        
 3   StoreCode        16433 non-null  object        
 4   StoreName        16433 non-null  object        
 5   Product Code     16433 non-null  object        
 6   Product Name     16433 non-null  object        
 7   MSL              16433 non-null  object        
 8   OSA Satus        16433 non-null  object        
 9   Quantity         16433 non-null  object        
 10  Amount(INR)      16433 non-null  object        
 11  Cost(INR)        11223 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(10)
memory usage: 1.5+ MB


### BA Sales2

In [46]:
# Connect and fetch into DataFrame 
try:
    conn = pyodbc.connect(conn_str)
    print("✅ Connected to SQL Server")

    # Define your query
    query = ''' SELECT
       [VisitDatetime] AS [Visit Date Time]
      ,[UserName] AS [Emp User ID]
      ,[FullName] AS [Employee Name]
      ,[LISStoreCode] AS [StoreCode]
      ,[StoreName]
      ,[CustomersContacted] AS [Customer Approached]
      ,[CustomersConverted] AS [Customer Converted]
      ,[ProductCategoryCode] AS [Product Code]
      ,[ProductCategoryName] AS [Product Name]
      ,CASE 
          WHEN TRY_CAST([SalesQty] AS float) IS NOT NULL AND TRY_CAST([SalesQty] AS float) != 0
          THEN TRY_CAST([SalesValue] AS float) / TRY_CAST([SalesValue] AS float)
        ELSE NULL
        END AS [UnitCost(MRP)]
      ,[SalesQty] AS [Sales Qty]
      ,[SalesValue] AS [Sales Amount]
  FROM [PowerBI_LISV2].[PidiliteManagement].[BASalesCase2ReportExport]
                WHERE  [Month] = MONTH(GETDATE())
                       AND year=YEAR(GETDATE())
                       and [UserName] NOT LIKE '%SUP%' 
                       AND [UserName] NOT LIKE '%test%'
                       AND [UserName] NOT LIKE 'TEST%'
    '''
    # Fetch results into a DataFrame
    BASales2 = pd.read_sql(query, conn)
    # Close the connection
    conn.close()
except Exception as e:
    print(f"❌ Error: {e}")

✅ Connected to SQL Server


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_20424\2096333908.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  BASales2 = pd.read_sql(query, conn)


In [47]:
BASales2.head()

Visit Date Time  Emp User ID Employee Name     StoreCode      StoreName  \
0 2025-05-01 23:30:52  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   
1 2025-05-01 23:30:52  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   
2 2025-05-01 23:30:52  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   
3 2025-05-01 23:30:52  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   
4 2025-05-01 23:30:52  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   

  Customer Approached Customer Converted      Product Code    Product Name  \
0                  76                 46  FevicrylRange-01  Fevicryl Range   
1                  76                 46  FevicrylRange-01  Fevicryl Range   
2                  76                 46       Fevistik-01        Fevistik   
3                  76                 46  FevicrylRange-01  Fevicryl Range   
4                  76                 46        SMPacks-01        SM Packs   

   UnitCost(MRP) Sales Qty Sales Amount  
0            1.0         1       150.00  
1            1.0         1       225.00  
2            1.0         2        70.00  
3            1.0         1       250.00  
4            1.0         1        50.00

In [49]:
BASales2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3245 entries, 0 to 3244
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Visit Date Time      3245 non-null   datetime64[ns]
 1   Emp User ID          3245 non-null   object        
 2   Employee Name        3245 non-null   object        
 3   StoreCode            3245 non-null   object        
 4   StoreName            3245 non-null   object        
 5   Customer Approached  3245 non-null   object        
 6   Customer Converted   3245 non-null   object        
 7   Product Code         3245 non-null   object        
 8   Product Name         3245 non-null   object        
 9   UnitCost(MRP)        2117 non-null   float64       
 10  Sales Qty            3208 non-null   object        
 11  Sales Amount         3208 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(10)
memory usage: 304.3+ KB


In [50]:
BASales2.to_csv(r'C:\Users\Administrator\LIS\Pidilite\BASales2.csv', index=False)

### Promotion Data

In [53]:
# Connect and fetch into DataFrame -
try:
    conn = pyodbc.connect(conn_str)
    print("✅ Connected to SQL Server")

    # Define your query
    query = ''' SELECT 
       [SubmittedDate&Time] AS [Visit DateTime]
      ,[Username] AS [Emp User ID]
      ,[FullName] AS [Employee Name]
      ,[LISStoreCode] AS [StoreCode]
      ,[StoreName] AS [StoreName]
      ,[ProductCode] AS [Promotion Product Code]
      ,[ProductName] AS [Promotion Product Name]
      ,[ProductBrandName] AS [Product Brand]
      ,[IsSamePromotionRunning] AS [Promotion is running on POS]
      ,[SameonPOS] AS [Is Promotion same]
      ,[PromotionType] AS [Type Of Promotion]
      , ' ' AS [Off on Shelf talker]
      , ' ' AS [Off on Product]
      , ' ' AS [Pack Offer]
      ,[PhotoLink] AS [Photopath]

  FROM [PowerBI_LISV2].[PidiliteManagement].[PromotionTrackingReportExport]

                WHERE  [Month] = MONTH(GETDATE())
                       AND year=YEAR(GETDATE())
                       and [UserName] NOT LIKE '%SUP%' 
                       AND [UserName] NOT LIKE '%test%'
                       AND [UserName] NOT LIKE 'TEST%'
    '''
    # Fetch results into a DataFrame
    PromotionData = pd.read_sql(query, conn)
    # Close the connection
    conn.close()
except Exception as e:
    print(f"❌ Error: {e}")

✅ Connected to SQL Server


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_20424\3791318535.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  PromotionData = pd.read_sql(query, conn)


In [54]:
PromotionData.head()

Visit DateTime  Emp User ID Employee Name     StoreCode      StoreName  \
0 2025-05-01 23:04:56  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   
1 2025-05-01 23:04:56  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   
2 2025-05-01 23:04:56  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   
3 2025-05-01 23:04:56  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   
4 2025-05-01 23:04:56  PIDI-ISP-42     Prabath S  Pidilite-019  LULU_PALAKKAD   

  Promotion Product Code                      Promotion Product Name  \
0      P-FFS801800801P2P                       FEVISTIK POWER [8 GM]   
1      P-FFS8009026000OP            FEVISTIK[FEVISTIK 25G]Fevistik50   
2      P-FFS800201500000  FEVISTIK SUPER GLU STICK 15GMFEVISTIK40 []   
3      P-FMR803120100001          FEVICOL MR [200 G] NEW ART SQUEEZY   
4      P-FKA802415000P2P    RNGL.CRTVE.CLAY BLPK ILOVEANIMALS 150G N   

  Product Brand Promotion is running on POS Is Promotion same  \
0      Fevistik                         Yes                No   
1      Fevistik                         Yes                No   
2      Fevistik                         Yes                No   
3       Fevicol                         Yes                No   
4      Rangeela                         Yes                No   

  Type Of Promotion Off on Shelf talker Off on Product Pack Offer Photopath  
0              None                                                          
1              None                                                          
2              None                                                          
3              None                                                          
4              None

In [71]:
PromotionData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10399 entries, 0 to 10398
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Visit DateTime               10399 non-null  datetime64[ns]
 1   Emp User ID                  10399 non-null  object        
 2   Employee Name                10399 non-null  object        
 3   StoreCode                    10399 non-null  object        
 4   StoreName                    10399 non-null  object        
 5   Promotion Product Code       10399 non-null  object        
 6   Promotion Product Name       10399 non-null  object        
 7   Product Brand                10399 non-null  object        
 8   Promotion is running on POS  10399 non-null  object        
 9   Is Promotion same            10399 non-null  object        
 10  Type Of Promotion            232 non-null    object        
 11  Off on Shelf talker          10399 non-nu

In [ ]:
PromotionData.to_csv(r'C:\Users\Administrator\LIS\Pidilite\PromotionData.csv', index=False)

### Coverage Calculations

In [19]:
# Connect and fetch into DataFrame - Extrating Data from ISPStoreAttendance_NormalExport ( Test & Supervisor Filtered)
try:
    conn = pyodbc.connect(conn_str)
    print("✅ Connected to SQL Server")

    # Define your query
    query = ''' SELECT USERNAME AS [Emp User ID],
                       FullName AS [Employee Name],
                       LISStoreCode AS StoreCode,
                       StoreName,
                       TRY_CONVERT(DATE, VisitDate,105) AS [Visit Date],
                       [CheckIn-Time] AS [StartDateTime],
                       [CheckOut-Time] AS [EndDateTime],
                       TimeDuration AS [NoOfWorkingHrs],
                       'Y' AS Covered,
                       CASE WHEN NoPermissionReason is null THEN 'Y'
                            ELSE 'N' 
                       END AS Merchandised,
                       NoPermissionReason AS [No Permission Reason],
                       PJPStatus AS [PJP Type]
                 
                FROM [PowerBI_LISV2].[PidiliteManagement].[ISPStoreAttendance_NormalExport]
                where  [Month] = MONTH(GETDATE())
                AND year=YEAR(GETDATE())
                and [UserName] NOT LIKE '%SUP%' 
            AND [UserName] NOT LIKE '%test%'
            AND [UserName] NOT LIKE 'TEST%'

    '''

    # Fetch results into a DataFrame
    CoveredPJP = pd.read_sql(query, conn)

    # Close the connection
    conn.close()



except Exception as e:
    print(f"❌ Error: {e}")

✅ Connected to SQL Server


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_4516\2399517793.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  CoveredPJP = pd.read_sql(query, conn)


In [20]:
CoveredPJP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Emp User ID           463 non-null    object
 1   Employee Name         463 non-null    object
 2   StoreCode             463 non-null    object
 3   StoreName             463 non-null    object
 4   Visit Date            463 non-null    object
 5   StartDateTime         463 non-null    object
 6   EndDateTime           463 non-null    object
 7   NoOfWorkingHrs        463 non-null    object
 8   Covered               463 non-null    object
 9   Merchandised          463 non-null    object
 10  No Permission Reason  0 non-null      object
 11  PJP Type              463 non-null    object
dtypes: object(12)
memory usage: 43.5+ KB


In [21]:
# Creating Key for CoveredPJP
CoveredPJP['Visit Date'] = pd.to_datetime(CoveredPJP['Visit Date'], format='%Y-%d-%m', errors='coerce')
CoveredPJP['Visit Date'] = CoveredPJP['Visit Date'].dt.strftime('%d/%m/%Y')
CoveredPJP['Key']=CoveredPJP['Visit Date']+"X"+CoveredPJP['Emp User ID']+"X"+CoveredPJP['StoreCode']
CoveredPJP['Key']

0       01/05/2025XPIDI-ISP-42XPidilite-019
1        01/05/2025XPIDI-ISP-06XPidilite-38
2        01/05/2025XPIDI-ISP-88XPidilite-58
3        01/05/2025XPIDI-ISP-86XPidilite-86
4        01/05/2025XPIDI-ISP-74XPidilite-32
                       ...                 
458      18/05/2025XPIDI-ISP-05XPidilite-37
459     18/05/2025XPIDI-ISP-09XPidilite-011
460      18/05/2025XPIDI-ISP-91XPidilite-88
461     18/05/2025XPIDI-ISP-102XPidilite-85
462    18/05/2025XPIDI-ISP-110XPidilite-002
Name: Key, Length: 463, dtype: object

In [22]:
PlannedPJP.head()

Emp User ID Employee Name     StoreCode  \
0   PIDI-ISP-77   Seejamol SM   Pidilite-66   
1  PIDI-ISP-114  Syam Mohan M  Pidilite-012   
2   PIDI-ISP-65      Srisha S  Pidilite-101   
3  PIDI-ISP-109     Soundarya  Pidilite-013   
4   PIDI-ISP-76      Sinchana  Pidilite-003   

                                   StoreName Visit Date  
0                       BB-TRIVANDRUM-OLD KA   1/5/2025  
1                            LULU_TRIVANDRUM   1/5/2025  
2  Reliance Retail-Bhartiya City Thanisandra   1/5/2025  
3                            LULU_COIMBATORE   1/5/2025  
4                     METRO Yashwantpur, Blr   1/5/2025

In [23]:
# Creating Key for PlannedPJP
PlannedPJP['Visit Date'] = pd.to_datetime(PlannedPJP['Visit Date'], dayfirst=True, errors='coerce')
PlannedPJP['Visit Date'] = PlannedPJP['Visit Date'].dt.strftime('%d/%m/%Y')

PlannedPJP['Key']=PlannedPJP['Visit Date']+"X"+PlannedPJP['Emp User ID']+"X"+PlannedPJP['StoreCode']
PlannedPJP['Key']

0        01/05/2025XPIDI-ISP-77XPidilite-66
1      01/05/2025XPIDI-ISP-114XPidilite-012
2       01/05/2025XPIDI-ISP-65XPidilite-101
3      01/05/2025XPIDI-ISP-109XPidilite-013
4       01/05/2025XPIDI-ISP-76XPidilite-003
                       ...                 
924    26/05/2025XPIDI-ISP-116XPidilite-015
925    28/05/2025XPIDI-ISP-116XPidilite-015
926    29/05/2025XPIDI-ISP-116XPidilite-015
927    30/05/2025XPIDI-ISP-116XPidilite-015
928    31/05/2025XPIDI-ISP-116XPidilite-015
Name: Key, Length: 929, dtype: object

In [24]:
PlannedPJP.head()

Emp User ID Employee Name     StoreCode  \
0   PIDI-ISP-77   Seejamol SM   Pidilite-66   
1  PIDI-ISP-114  Syam Mohan M  Pidilite-012   
2   PIDI-ISP-65      Srisha S  Pidilite-101   
3  PIDI-ISP-109     Soundarya  Pidilite-013   
4   PIDI-ISP-76      Sinchana  Pidilite-003   

                                   StoreName  Visit Date  \
0                       BB-TRIVANDRUM-OLD KA  01/05/2025   
1                            LULU_TRIVANDRUM  01/05/2025   
2  Reliance Retail-Bhartiya City Thanisandra  01/05/2025   
3                            LULU_COIMBATORE  01/05/2025   
4                     METRO Yashwantpur, Blr  01/05/2025   

                                    Key  
0    01/05/2025XPIDI-ISP-77XPidilite-66  
1  01/05/2025XPIDI-ISP-114XPidilite-012  
2   01/05/2025XPIDI-ISP-65XPidilite-101  
3  01/05/2025XPIDI-ISP-109XPidilite-013  
4   01/05/2025XPIDI-ISP-76XPidilite-003

In [25]:
# PlannedPJP till Day-1
today = pd.to_datetime(datetime.today().date())
PlannedPJP['Visit Date'] = pd.to_datetime(PlannedPJP['Visit Date'], dayfirst=True, errors='coerce')


In [26]:

PlannedPJP = PlannedPJP[PlannedPJP['Visit Date'] <= today - pd.Timedelta(days=1)]

# Filter keys in PlannedPJP that are NOT in CoveredPJP
PlannedPJP_uncovered = PlannedPJP[~PlannedPJP['Key'].isin(CoveredPJP['Key'])]

PlannedPJP_uncovered=PlannedPJP_uncovered[["Emp User ID", "Employee Name","StoreCode","StoreName","Visit Date"]]

In [27]:
PlannedPJP_uncovered['StartDateTime'] = ''
PlannedPJP_uncovered['EndDateTime'] = ''
PlannedPJP_uncovered['NoOfWorkingHrs'] = ''
PlannedPJP_uncovered['Covered'] = 'N'
PlannedPJP_uncovered['Merchandised'] = 'N'
PlannedPJP_uncovered['No Permission Reason'] = ''
PlannedPJP_uncovered['PJP Type'] = 'Planned'
PlannedPJP_uncovered['Visit Date'] = pd.to_datetime(PlannedPJP_uncovered['Visit Date'], dayfirst=True, errors='coerce')
PlannedPJP_uncovered['Visit Date'] = PlannedPJP_uncovered['Visit Date'].dt.strftime('%d/%m/%Y')

#Creating Final DF
PJPCoverage = pd.concat([CoveredPJP, PlannedPJP_uncovered], ignore_index=True)
PJPCoverage = PJPCoverage.drop('Key', axis=1)


In [28]:
# #Sending df on sheet
# Coverage_sh.batch_clear(["A:AX"])
# set_with_dataframe(Coverage_sh, FinalPJP)

In [29]:
PJPCoverage.to_csv(r'C:\Users\Administrator\LIS\Pidilite\PJPCoverage.csv', index=False)